In [1]:
import numpy as np
import tensorflow as tf

from scada_forecast.preprocess import read_humidity, read_temperature, read_scada
from scada_forecast.preprocess import merge_dataframes, add_calendar_features, prepare_data

from scada_forecast.model import create_model, train_model, inference

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
df_humidity = read_humidity('data/scada/DoAm.xlsx')
df_temperature = read_temperature('data/NhietDoQuaKhu.xlsx')
df_scada = read_scada('data/scada/Dữ liệu SCADA Phụ tải 26.08.2020.xlsx')

In [3]:
df = merge_dataframes(df_scada, df_temperature, df_humidity)

In [5]:
df.columns

Index(['Độ ẩm B.M. Thuột', 'Độ ẩm Bãi Cháy', 'Độ ẩm Bạch. L.Vĩ',
       'Độ ẩm Cao Bằng', 'Độ ẩm Cà Mau', 'Độ ẩm Côn Đảo', 'Độ ẩm Cần Thơ',
       'Độ ẩm Hoài Nhơn', 'Độ ẩm Huế', 'Độ ẩm Hà Giang', 'Độ ẩm Hà Tĩnh',
       'Độ ẩm Hà Đông', 'Độ ẩm Hòa Bình', 'Độ ẩm Hồi Xuân', 'Độ ẩm Kỳ Anh',
       'Độ ẩm Lào Cai', 'Độ ẩm Lạng Sơn', 'Độ ẩm Móng Cái', 'Độ ẩm Mường Lay',
       'Độ ẩm Nam Định', 'Độ ẩm Nha Trang', 'Độ ẩm Nhà Bè', 'Độ ẩm Phan Rang',
       'Độ ẩm Phan Thiết', 'Độ ẩm Phù Liễn', 'Độ ẩm Phú Quốc', 'Độ ẩm Phú Qúy',
       'Độ ẩm Plâycu', 'Độ ẩm Q.Ngãi', 'Độ ẩm Quy Nhơn', 'Độ ẩm Rạch Gía',
       'Độ ẩm Sơn La', 'Độ ẩm T.Quang', 'Độ ẩm Tam Đường', 'Độ ẩm Thanh Hóa',
       'Độ ẩm Tiên Yên', 'Độ ẩm Trường Sa', 'Độ ẩm Tuy Hòa', 'Độ ẩm Vinh',
       'Độ ẩm Vũng Tàu', 'Độ ẩm Điện Biên', 'Độ ẩm Đà Nẵng', 'Độ ẩm Đồng Hới',
       'Nhiệt độ Tam Đường', 'Nhiệt độ Mường Lay(Lai Châu)',
       'Nhiệt độ Điện Biên', 'Nhiệt độ Sơn La', 'Nhiệt độ Hòa Bình',
       'Nhiệt độ Lào Cai', 'Nhiệt đ

In [ ]:
df = add_calendar_features(df, use_lunar=True, use_holiday=False)

In [ ]:
hour_steps = int(60 / 5)
df, features_dict, categorical_cols, numeric_cols, target_col = prepare_data(df, 
                                                                             hour_steps=hour_steps, 
                                                                             use_temp=True, 
                                                                             use_humidity=True, 
                                                                             use_lunar=True,
                                                                             use_holiday=False)

In [ ]:
dtype_dict = {col: df[col].dtype for col in categorical_cols + numeric_cols}

In [ ]:
categorical_unique_values_dict = {
    'Month': list(range(1, 12 + 1)),
    'DayOfYear': list(range(1, 365 + 1)),
    'DayOfWeek': list(range(7)),
    'Hour': list(range(24)),
    'LunarMonth': list(range(1, 12 + 1)),
    'LunarDayOfMonth': list(range(1, 30 + 1)),
    'HolidayCat': list(range(12))
}

In [ ]:
n_val = 100 * 24 * hour_steps # 100 days
n_test = 100 * 24 * hour_steps # 100 days
input_width = hour_steps
chosen_features = (features_dict['calendar_features'] + features_dict['lag_features'] + 
                   features_dict['temperature_features'] + features_dict['humidity_features'])
ckpt_path = 'tmp/ckpt'

In [ ]:
train_model(df, input_width, n_val, n_test, hour_steps,
            dtype_dict, categorical_unique_values_dict, 
            chosen_features, categorical_cols, 
            numeric_cols, target_col, ckpt_path)

In [ ]:
forecast_df = inference(df, input_width, n_test, hour_steps, ckpt_path,
                        dtype_dict, categorical_unique_values_dict, 
                        chosen_features, categorical_cols, numeric_cols, target_col)